In [ ]:
import k3d
import numpy as np
import numpy.typing as npt
from scipy.spatial import Delaunay

In [ ]:
def create_grid(size: int) -> npt.NDArray[np.integer]:
    pos = np.zeros((size**2, 3))
    x = np.arange(size)
    y = np.arange(size)
    xx, yy = np.meshgrid(x, y)
    pos[:, :2] = np.array((xx.ravel(), yy.ravel())).T
    return pos


def create_height_data(size: int) -> npt.NDArray[np.float32]:
    z_matrix = np.zeros((size, size))
    for _ in range(100):
        width = int(0.4 * size * np.random.rand())
        peak_size = int(0.02 * size * np.random.rand())
        i, j = np.random.randint(0, size - width, size=2)
        z_matrix[i : i + width, j : j + width] += create_gaussian_peak(
            width=width, height=peak_size, size=width
        )
    for _ in range(200):
        width = int(0.4 * size * np.random.rand())
        peak_size = int(0.01 * size * np.random.rand())
        i, j = np.random.randint(0, size - width, size=2)
        z_matrix[i : i + width, j : j + width] += create_gaussian_peak(
            width=width, height=peak_size, size=width
        )
    return z_matrix


def create_gaussian_peak(
    height: float, width: float, size: int = 5
) -> npt.NDArray[np.float32]:
    # size preferably an odd number
    center = int(size / 2)
    grid = create_grid(size=size)[:, :2]
    d = np.linalg.norm(grid - center, axis=1)
    gaussian = height * np.exp(-(d**2) / (2 * width))
    return gaussian.reshape(size, size)


def create_mesh_indices(size: int):
    row_indices = []

    for i in range(0, size - 1, 1):
        row_indices.append([i, i + 1, i + size])
        row_indices.append([i + 1, i + size + 1, i + size])

    row_indices = np.array(row_indices)
    indices = np.zeros(((size - 1) ** 2 * 2, 3))
    for i in range(0, size - 1):
        indices[i * (size - 1) * 2 : (i + 1) * (size - 1) * 2] = row_indices + i * size
    return indices

In [ ]:
grid_size = 3200
grid = create_grid(grid_size)
z = create_height_data(grid_size)
indices = create_mesh_indices(grid_size)
grid[:, 2] = z.ravel()

In [ ]:
plot = k3d.plot(grid_visible=False, height=1000)

plot += k3d.mesh(
    vertices=grid, indices=indices, side="double", wireframe=False, flat_shading=True
)
# plot += k3d.points(grid, color=int("FF0000", 16), point_size=0.5)
plot